# Begin Data Processing
## MP AP

In [1]:
import torch
import numpy as np
import re

import pandas as pd
import os

######################################
# Settings
current_Q = 1
dataset_name = 'mnist'
# Data Source
source_name = 'test_data_by_attack'
# attacks that need to be evaluated
Attack_ID = ['BLI','DLI','DS','NS','No_Attack','No_Attack_nsds']
ALL_Attack = ['BatchLabelReconstruction','DirectLabelScoring','DirectionbasedScoring','NormbasedScoring',\
                'No_Attack','No_Attack_nsds'] 
######################################
#Attack_ID = ['BLI','DLI','DS','LRB','PMC','AMC','GRN','MF','NoAttack','NoAttack_nsds','NS','NSB','TBM']
# ALL_Attack = ['ReplacementBackdoor','MissingFeature','NoisySample',\
#               'BatchLabelReconstruction','DirectLabelScoring','DirectionbasedScoring','NormbasedScoring',\
#               'PassiveModelCompletion','ActiveModelCompletion',\
#               'ResSFL','GenerativeRegressionNetwork',\
#                 'No_Attack','No_Attack_nsds']  



final_columns = ['K','bs','LR','num_class','Q','top_trainable','epochs','Defense_name',\
                'attack_name','attack_param',\
                'Main Task Accuracy','Attack Performance']
summary = pd.DataFrame(columns=final_columns) 

def get_avg(total_df,trainable,Q,Defense):
    '''
    2.0, 1024.0, 0.01, 10.0, 1.0, 0.0, 30.0, 'BatchLabelReconstruction', '0.05',0.8874, 0.129883 ,CAE_0.0
    0       1      2     3    4    5     6           7                     8     9mp      10ap     11 
    '''
    if len(total_df) == 0:
        print(Defense,': none file')
        return
    basic_info = list(total_df.iloc[0])[:7]  # 0-6
    
    assert Defense == list(total_df.iloc[0])[11] # 11
    attack_name =list(total_df.iloc[0])[7]
    attack_param = list(total_df.iloc[0])[8] 
    
    mp = total_df['main_acc'].mean()
    if attack_name in ['MissingFeature','NoisySample']:
        if max(list(total_df['attack_metric']))<=0:
            ap = 0
        else:
            ap_list = []
            for _ap in list(total_df['attack_metric']):
                if _ap >=0:
                    ap_list.append(_ap)
            ap = sum(ap_list)/len(ap_list)
    else:
        ap = total_df['attack_metric'].mean()
    
    _info = basic_info+ [Defense] +[attack_name,attack_param]+[mp,ap]

    summary.loc[len(summary.index)] = _info
    

for attack_id in Attack_ID:
    attack_data_dir = source_name+'/'+attack_id+'/'
    print(attack_data_dir)

    ########## READ FROM ################
    for trainable in ['0']: # ,'1'
        data_dir = attack_data_dir+'Q'+str(current_Q)+'/'+trainable+'/'
        filenames=os.listdir(data_dir) # attacks

        ignore_pattern = re.compile(r'ignore')
        # For Each Defense
        for file_name in filenames:
            defense_name = file_name.split(',')[0]
            # print('Defense:',defense_name)

            columns = ['K','bs','LR','num_class','Q','top_trainable','epochs','attack_name','attack_param',\
                    'main_acc','attack_metric']
            df = pd.DataFrame(columns=columns) 
            f=open(data_dir+file_name, encoding='gbk')

            for line in f:
                if line == '' or line == '\n' or line[0]=='=':
                    continue
                
                if (ignore_pattern.search(line)):
                    continue
                
                info = line.split(',')

                if len(info) == 1:
                    print('Defense:',defense_name,attack_id)
                    print(line)
                    assert 1>2

                info = info[1].strip()
                final = info.split('|')
                for index in range(7):
                    final[index] = float(final[index])
                final[9]= float(final[9])

                # Some Specific Process for each attack
                if (final[7]=='No_Attack'):
                    
                    if int(final[3]) == 2:
                        final[7]='No_Attack_nsds'

                if len(final) == 12: 
                    if (final[7]=='GenerativeRegressionNetwork') or (final[7]=='ResSFL'):
                        randmse =  float(final[10])
                        mse = float(final[11])
                        final[10] = 1-mse #(randmse-mse)/randmse
                        final = final[:-1]
                    elif (final[7]=='DirectionbasedScoring') or (final[7]=='NormbasedScoring'):
                        acc = float(final[10])
                        auc = float(final[11])
                        if (final[7]=='NormbasedScoring'):
                            acc = max(acc,1-acc) 
                        final[10] = acc
                        final = final[:-1]
                    else:
                        print('error:')
                        print(final)
                else:
                    final[10]= float(final[10])
                
                if final[10]<0:  # AP should be >0 
                    final[10]=0

                df.loc[len(df.index)] = final
            # add defense name
            df.insert(df.shape[1], 'DefenseName', defense_name)
            
            # get avg info for each attack
            get_avg(df,int(trainable),current_Q,defense_name) # add averaged info into summary



test_data_by_attack/BLI/
test_data_by_attack/DLI/
test_data_by_attack/DS/
test_data_by_attack/NS/
test_data_by_attack/No_Attack/
test_data_by_attack/No_Attack_nsds/


In [2]:
raw_file_name = 'record/'+dataset_name+'_'+str(current_Q)+'_raw_record.xlsx'
summary.to_excel(raw_file_name)
print(raw_file_name)

record/mnist_1_raw_record.xlsx


# Load Metrcis: Get DCS for each defense-attack pair

### DCS

In [3]:
import math
TARGETED_BACKDOOR = ['ReplacementBackdoor'] 
UNTARGETED_BACKDOOR = ['NoisySample','MissingFeature'] # df2
LABEL_INFERENCE = ['BatchLabelReconstruction','DirectLabelScoring','NormbasedScoring','DirectionbasedScoring',\
                   'PassiveModelCompletion','ActiveModelCompletion'] # label_recovery
FEATURE_RECONSTRUCTION = ['GenerativeRegressionNetwork','ResSFL'] # 

 

DEFENSE = ['MID_Active_1e-08','MID_Active_0.0','MID_Active_0.0001', \
'MID_Active_0.1','MID_Active_0.01','MID_Active_1e-06','MID_Active_1.0',  'MID_Active_100', 'MID_Active_10000',  \

'GradPerturb_0.01', 'GradPerturb_0.1',  'GradPerturb_1.0','GradPerturb_10.0',\

'DistanceCorrelation_0.0001', 'DistanceCorrelation_0.01', 'DistanceCorrelation_0.1', 'DistanceCorrelation_0.3',\

'DCAE_1.0','DCAE_0.5','DCAE_0.1','DCAE_0.0',  \
'CAE_0.0','CAE_0.5','CAE_0.1','CAE_1.0', \

'GradientSparsification_95.0','GradientSparsification_97.0',\
'GradientSparsification_99.0','GradientSparsification_99.5',\

'LaplaceDP_0.0001', 'LaplaceDP_0.001','LaplaceDP_0.01','LaplaceDP_0.1', \
'GaussianDP_0.0001','GaussianDP_0.01','GaussianDP_0.1', 'GaussianDP_0.001',\
'None_None']

################### Change Here ###################
raw_file_name = 'record/'+dataset_name+'_'+str(current_Q)+'_raw_record.xlsx'
summary=pd.read_excel(raw_file_name) #'exp_result/'+load_name+'_raw_record.xlsx')
print('Load Data from:',raw_file_name)
################### Change Here ###################

final_columns = ['K','bs','LR','num_class','Q','top_trainable','epochs','mp','ap','Defense_name','Attack_name','DCS']
report = pd.DataFrame(columns=final_columns) 


def cal_dcs(mp,ap,mp0,ap0,beta=0.5):
    d_ap = max(ap-ap0,0) # ap0 should < ap
    d_mp = max(mp0-mp,0) # mp0 should > mp
    distance= math.sqrt((1-beta)*(d_ap**2)+beta*(d_mp**2))
    dsc = 1/(1+distance)
    return dsc


def get_dcs(total_df,trainable,Q,defense_name,attack_name):
    '''
    total_df: summary
    'K', 'bs', 'LR', 'num_class', 'Q', 'top_trainable', 'epochs',
       'Defense_name', 'attack_name', 'attack_param', 'Main Task Accuracy',
       'Attack Performance'
    '''
    _df= total_df.loc[(total_df['attack_name']==attack_name) & (total_df['Defense_name']==defense_name) & (total_df['top_trainable']==trainable) & (total_df['Q']==Q)]

    if len(_df) == 0:
        print(defense_name,attack_name,'none file')
        return
    mp=list(_df['Main Task Accuracy'])[0] 
    ap = list(_df['Attack Performance'])[0]
    if len(list(_df['Main Task Accuracy'])) != 1:
        print(defense_name,' ',attack_name,' ',list(_df['Main Task Accuracy']))
    basic_info = list(_df.iloc[0][1:8])


    ###### Ideal Performance #######
    if attack_name in ['DirectionbasedScoring','NormbasedScoring','No_Attack_nsds']:
        vanilla = total_df.loc[(total_df['attack_name']=='No_Attack_nsds') &(total_df['Defense_name']=='None_None') & (total_df['top_trainable']==trainable) & (total_df['Q']==Q)]
    else:
        vanilla = total_df.loc[(total_df['attack_name']=='No_Attack') &(total_df['Defense_name']=='None_None') & (total_df['top_trainable']==trainable) & (total_df['Q']==Q)]
    if len(list(vanilla['Main Task Accuracy']))==0:
        print(attack_name,'missing corresponding vannila case')
        assert 1>2
    mp0 = list(vanilla['Main Task Accuracy'])[0]
    ap0 = 0
    ###### Ideal Performance #######

    dcs = cal_dcs(mp,ap,mp0,ap0,beta=0.5)

    _info = basic_info+[mp,ap]+[defense_name]+[attack_name]+[dcs]
    report.loc[len(report.index)] = _info

for defense_name in DEFENSE:
    for attack_name in ALL_Attack:
        # print(defense_name,attack_name)
        # get_avg for this defense
        get_dcs(summary,0,current_Q,defense_name,attack_name)
        #get_dcs(summary,1,current_Q,defense_name,attack_name)



Load Data from: record/mnist_1_raw_record.xlsx


In [4]:
dcs_report_name = 'record/'+dataset_name+'_'+str(current_Q)+'_dcs.xlsx'
report.to_excel(dcs_report_name,index=False)

print(dcs_report_name)

record/mnist_1_dcs.xlsx


# Figures

In [5]:

brief_dict={
    'ReplacementBackdoor':'TB-LRB',
    'MissingFeature':'NTB-MF',
    'NoisySample':'NTB-NSB',
    'BatchLabelReconstruction':'LI-BLI',
    'DirectLabelScoring':'LI-DLI',
    'DirectionbasedScoring':'LI-DS',
    'NormbasedScoring':'LI-NS',
    'PassiveModelCompletion':'LI-PMC',
    'ActiveModelCompletion':'LI-AMC',
    'GenerativeRegressionNetwork':'FR-GRN',
    'ResSFL':'FR-TBM',
    'No_Attack':'w/o attack'
}
TARGETED_BACKDOOR = ['ReplacementBackdoor'] 
UNTARGETED_BACKDOOR = ['NoisySample','MissingFeature'] # df2
LABEL_INFERENCE = ['PassiveModelCompletion','BatchLabelReconstruction','DirectLabelScoring','NormbasedScoring','DirectionbasedScoring',\
                   'ActiveModelCompletion'] # label_recovery
FEATURE_RECONSTRUCTION = ['GenerativeRegressionNetwork','ResSFL']
######## 各类Attack 二维散点 #######
import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
report1 = pd.read_excel('record/mnist_1_dcs.xlsx')
#report5 = pd.read_excel('exp_result/record/mnist_5_dcs.xlsx')

marker_list = ['P','X','D','^','o','1','H','2','h','s']
# 从小到大
CAE = ['CAE_0.0','CAE_0.1','CAE_0.5','CAE_1.0']
DCAE = ['DCAE_0.0','DCAE_0.1','DCAE_0.5','DCAE_1.0']

Gaussian = ['GaussianDP_0.0001','GaussianDP_0.001','GaussianDP_0.01','GaussianDP_0.1']
Laplace = ['LaplaceDP_0.0001','LaplaceDP_0.001','LaplaceDP_0.01','LaplaceDP_0.1']
GP = ['GradPerturb_0.01','GradPerturb_0.1','GradPerturb_1.0','GradPerturb_10.0']

MID_Active=['MID_Active_0.0','MID_Active_1e-08','MID_Active_1e-06','MID_Active_0.0001','MID_Active_0.01','MID_Active_0.1',\
    'MID_Active_1.0','MID_Active_100','MID_Active_10000']
GS = ['GradientSparsification_95.0','GradientSparsification_97.0',\
    'GradientSparsification_99.0','GradientSparsification_99.5']
dcor = ['DistanceCorrelation_0.0001','DistanceCorrelation_0.01','DistanceCorrelation_0.1','DistanceCorrelation_0.3']

NoDefense = ['None_None']
Defense_dict = {
    'G-DP':Gaussian,
    'L-DP':Laplace,
    'GS':GS,
    'GPer':GP,
    
    'dCor':dcor,
    'CAE':CAE,
    'DCAE':DCAE,
    
    'MID':MID_Active,
    
    'None': NoDefense
}# '

#'#eed777'
colors = ['#3682be','#f05326','#DAA520','#334f65','#df3881','#93c555','#844bb3','#b3974e']

# marker_list = ['o'(DP-G), 'v'(DP-L), '^'(GS), '2'(GPer), '.'(dCor), '4'(CAE), '+'(DCAE), 'h(MID)' ,'*'(ideal), 's'(w/o defense)]
# color_list = ['#1f77b4'(DP-G), '#ff7f0e'(DP-L), '#2ca02c'(GS), '#ad8150'(GPer), '#ff028d'(dCor), '#d62728'(CAE), '#bcbd22'(DCAE), '#9467bd'(MID)]

marker_list = ['o', 'v', '^', 'X', '.', '*', 'D', 'h' , 's']
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#ad8150', '#ff028d', '#d62728', '#bcbd22', '#9467bd']


APPLY_LEGEND = 0
def make_positive(origin):
    for i in range(len(origin)):
        if origin[i]<0:
            origin[i]=0
    return origin

def get2d_MPAP(mode,Q,attack_name,APPLY_LEGEND,idx):
    
    ########## Vanilla: No Attack [best MP]    random guess[worst AP]
    # if (attack_name in TARGETED_BACKDOOR) or (attack_name in LABEL_INFERENCE):
    #     ap0 = 0.1
    #     if attack_name in ['DirectionbasedScoring','NormbasedScoring']:
    #         ap0 = 0.5
    # else:
    #     ap0 = 0
    ap0 = 0
    if Q == 1:
        if attack_name in ['DirectionbasedScoring','NormbasedScoring']:
            vanilla_df = report1.loc[(report1['Attack_name']=='No_Attack_nsds') &(report1['top_trainable']==mode) & (report1['Defense_name']=='None_None')]
        else:
            vanilla_df = report1.loc[(report1['Attack_name']== 'No_Attack') &(report1['top_trainable']==mode) & (report1['Defense_name']=='None_None')]
    if len(list(vanilla_df['mp'])) == 0:
        print(attack_name , vanilla_df)
    mp0 = list(vanilla_df['mp'])[0]
    

    ax = plt.subplot(4,4,idx)
    i = 0
    for _name in Defense_dict.keys():
        defense_list = Defense_dict[_name]
        x = []
        y = []
        for _defense in defense_list:
            if Q == 1:
                sub_df = report1.loc[(report1['Attack_name']== attack_name) &(report1['top_trainable']==mode) & (report1['Defense_name']==_defense)]
            # else:
            #     sub_df = report5.loc[(report5['top_trainable']==mode) & (report5['Defense_name'].isin(defense_list))]
            Tmp = 'mp'
            Tap = 'ap'
            if len(list(sub_df[Tmp]))==0:
                continue
            
            x.append(list(sub_df[Tmp])[0])
            y.append(list(sub_df[Tap])[0])

        if len(x) == 0:
            print(attack_name,' ',_name,' not ready')
            continue

        scale = 95
        area = np.linspace(5*scale,35*scale,len(x))
        symbol_scale = 64
        symbol_area = np.linspace(5*symbol_scale,35*symbol_scale,len(x))
        
        x = make_positive(x)
        y = make_positive(y)
        ####### Points #########
        if _name == 'None': 
            ax.scatter(x, y,c='k', marker= 's',s= 190,label='w/o defense',  alpha = 1)  # center point
        
        else:
            #ax.scatter(x, y,c=colors[i],  s= area, alpha = 0.2)  
            ax.scatter(x, y,c=colors[i], marker= marker_list[i],s= symbol_area,label=_name,alpha = 0.6)  # 160 center point
            
            #ax.plot(x, y,c=colors[i],linestyle=':',linewidth=3,alpha = 0.6)  # line
            
        i = i+1
    ####### Vanilla #######
    # ax.scatter(mp0, ap0,c='r', s= 200,marker='*', alpha = 1,label='Ideal Performance')  # center point
    

    if attack_name == 'NormbasedScoring':
        ax.legend(fontsize=32)
    ax.set_title(brief_dict[attack_name],fontsize=48)
    ax.tick_params(labelsize=24)
    # fig_name = 'exp_result/figures/MPAP/'+load_name+'_'+attack_name+'.png'
    # print(fig_name)
    # plt.savefig(fig_name)
    # plt.close()

#################################
mode = 0 # agg
current_Q = 1
###################################
attack_types = ['TB','NTB','LI','FR']

plt.figure(dpi=300,figsize=(50,50))
idx = 1
for attack_name in ALL_Attack:
    if attack_name == 'No_Attack_nsds': # No attack for 2 classes
        continue
    get2d_MPAP(mode,current_Q,attack_name,APPLY_LEGEND,idx)
    APPLY_LEGEND = APPLY_LEGEND+1
    idx = idx+1


plt.savefig('figures/ALL.png')

# Final Metric: T-DCS and C-DCS

In [6]:
TARGETED_BACKDOOR = ['ReplacementBackdoor'] 
UNTARGETED_BACKDOOR = ['NoisySample','MissingFeature'] # df2
LABEL_INFERENCE = ['PassiveModelCompletion','BatchLabelReconstruction','DirectLabelScoring','NormbasedScoring','DirectionbasedScoring',\
                   'ActiveModelCompletion'] # label_recovery
FEATURE_RECONSTRUCTION = ['GenerativeRegressionNetwork','ResSFL'] #
dcs_report_name = 'record/'+dataset_name+'_'+str(current_Q)+'_dcs.xlsx'
print('Load Data From:',dcs_report_name)
report=pd.read_excel(dcs_report_name) 


def average(value):
    if len(value) == 0:
        return -1
    
    clean_value = []
    for i in range(len(value)):
        if value[i] != -1:
            clean_value.append(value[i])
    result = sum(clean_value)/len(clean_value)
    return result


final_columns = ['K','bs','LR','num_class','Q','top_trainable','epochs','Defense_name','Defense_Param',\
                    'T-DCS_LI','T-DCS_FR','T-DCS_TB','T-DCS_NTB','C-DCS'] # 
final_report = pd.DataFrame(columns=final_columns) 

def get_final(total_df,trainable,Q,defense_name):
    
    dca_list = []
    for attack_name in TARGETED_BACKDOOR:
        _df= total_df.loc[(total_df['Attack_name']==attack_name) & (total_df['Defense_name']==defense_name) & (total_df['top_trainable']==trainable) & (total_df['Q']==Q)]
        if len(_df) == 0:
            print(defense_name,attack_name,'none file')
            continue
        dca_list.append(list(_df['DCS'])[0])
 
        basic_info = list(_df.iloc[0][1:8])
    dcs_TB = average(dca_list)

    dca_list = []
    for attack_name in UNTARGETED_BACKDOOR:
        _df= report.loc[(total_df['Attack_name']==attack_name) & (total_df['Defense_name']==defense_name) & (total_df['top_trainable']==trainable) & (total_df['Q']==Q)]
        if len(_df) == 0:
            print(defense_name,attack_name,'none file')
            continue
        dca_list.append(list(_df['DCS'])[0])

        basic_info = list(_df.iloc[0][1:8])
    dcs_NTB = average(dca_list)
    
    dca_list = []
    for attack_name in LABEL_INFERENCE:
        _df= report.loc[(report['Attack_name']==attack_name) & (report['Defense_name']==defense_name) & (report['top_trainable']==trainable) & (report['Q']==Q)]
        if len(_df) == 0:
            print(defense_name,attack_name,'none file')
            continue
        dca_list.append(list(_df['DCS'])[0])

        basic_info = list(_df.iloc[0][1:8])
    dcs_LI = average(dca_list)
    
    dca_list = []
    for attack_name in FEATURE_RECONSTRUCTION:
        _df= report.loc[(report['Attack_name']==attack_name) & (report['Defense_name']==defense_name) & (report['top_trainable']==trainable) & (report['Q']==Q)]
        if len(_df) == 0:
            print(defense_name,attack_name,'none file')
            continue
        dca_list.append(list(_df['DCS'])[0])

        basic_info = list(_df.iloc[0][1:8])
    dcs_FR = average(dca_list)

    d_list = defense_name.split('_')
    d_name = d_list[0]
    d_para = d_list[-1]
    cdcs = average([dcs_TB,dcs_NTB,dcs_LI,dcs_FR])
    _info = basic_info+[d_name,d_para]+[dcs_LI,dcs_FR,dcs_TB,dcs_NTB,cdcs]

    final_report.loc[len(final_report.index)] = _info

for defense_name in DEFENSE:
    #print(defense_name)
    get_final(report,0,current_Q,defense_name)
    #get_final(report,1,current_Q,defense_name)

Load Data From: record/mnist_1_dcs.xlsx
MID_Active_1e-08 ReplacementBackdoor none file
MID_Active_1e-08 NoisySample none file
MID_Active_1e-08 MissingFeature none file
MID_Active_1e-08 PassiveModelCompletion none file
MID_Active_1e-08 ActiveModelCompletion none file
MID_Active_1e-08 GenerativeRegressionNetwork none file
MID_Active_1e-08 ResSFL none file
MID_Active_0.0 ReplacementBackdoor none file
MID_Active_0.0 NoisySample none file
MID_Active_0.0 MissingFeature none file
MID_Active_0.0 PassiveModelCompletion none file
MID_Active_0.0 ActiveModelCompletion none file
MID_Active_0.0 GenerativeRegressionNetwork none file
MID_Active_0.0 ResSFL none file
MID_Active_0.0001 ReplacementBackdoor none file
MID_Active_0.0001 NoisySample none file
MID_Active_0.0001 MissingFeature none file
MID_Active_0.0001 PassiveModelCompletion none file
MID_Active_0.0001 ActiveModelCompletion none file
MID_Active_0.0001 GenerativeRegressionNetwork none file
MID_Active_0.0001 ResSFL none file
MID_Active_0.1 Repl

In [7]:
cdcs_report_name = 'record/'+dataset_name+'_'+str(current_Q)+'_final_dcs.xlsx'
final_report.to_excel(cdcs_report_name)
print(cdcs_report_name)

record/mnist_1_final_dcs.xlsx
